In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [3]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [4]:
feature_list

{'SYN': ['ts',
  'IP_flags',
  'IP_DF',
  'TCP_dataofs',
  'TCP_SYN',
  'TCP_ACK',
  'sport_class',
  'dst_IP_diversity',
  'dst_port_diversity',
  'pck_size_sum_of_EW',
  'ts_diff',
  'ts_std_WE',
  'ts_sum_of_EW',
  'TCP_window_std_WE',
  'pck_size_mean_2',
  'ts_mean_2',
  'ts_std_2',
  'TCP_window_mean_2',
  'TCP_SYN_sum',
  'TCP_ACK_sum',
  'TCP_SYN_ratio',
  'TCP_ACK_SR',
  'ts_mean_6',
  'ts_std_6',
  'pck_size_mean_9',
  'ts_mean_9',
  'ts_std_9',
  'TCP_window_mean_9',
  'TCP_ACK_R',
  'Label'],
 'HTTP': ['ts',
  'TCP_flags',
  'dport_class',
  'pck_size_sum_of_EW',
  'ts_std_WE',
  'ts_sum_of_EW',
  'payload_bytes_sum_of_EW',
  'entropy_mean_WE',
  'TCP_window_mean_2',
  'sport_sum',
  'TCP_ACK_sum',
  'TCP_ACK_ratio',
  'sum',
  'TCP_ACK_SR',
  'ts_std_6',
  'TCP_window_mean_6',
  'TCP_PSH_R',
  'Label'],
 'ACK': ['payload_bytes_mean_WE',
  'sport_sum',
  'TCP_ACK_ratio',
  'TCP_ACK_SR',
  'ts_mean_6',
  'Label'],
 'UDP': ['IP_tos',
  'IP_DF',
  'IP_proto',
  'TCP_dataofs',


In [5]:
file_list={'SYN': ['./INPUT/SM/DoS-SYN-1.csv', './INPUT/VAL/VAL-SYN.csv'],
 'HTTP': ['./INPUT/SM/MB-HTTP-4.csv', './INPUT/VAL/VAL-HTTP.csv'],
 'ACK': ['./INPUT/SM/MB-ACK-4.csv', './INPUT/VAL/VAL-ACK.csv'],
 'UDP': ['./INPUT/SM/MB-UDP-4.csv', './INPUT/VAL/VAL-UDP.csv'],
 'ARP': ['./INPUT/SM/MitM-ARP-6.csv', './INPUT/VAL/VAL-ARP.csv'],
 'SP': ['./INPUT/SM/Scan-Port-3.csv', './INPUT/VAL/VAL-SP.csv'],
 'BF': ['./INPUT/SM/MB-BF-5.csv', './INPUT/VAL/VAL-BF.csv'],
 'OS': ['./INPUT/SM/Scan-OS-3.csv', './INPUT/VAL/VAL-OS.csv'],
 'SCHD': ['./INPUT/SM/Scan-HDis-3.csv', './INPUT/VAL/VAL-SCHD.csv'],
 'MHDis': ['./INPUT/SM/MB-HDis-3.csv', './INPUT/VAL/VAL-MHDis.csv']}

In [6]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

# RandomizedSearchCV  DT

In [26]:
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9998489480892206   3.873    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:38<00:00,  1.98s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  4.27     24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:52<00:00,  1.05s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.771    24      


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.125    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:35<00:00,  1.42it/s]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.847    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:25<00:00,  1.94it/s]

    criterion      max_depth    max_features    min_samples_split        F1    Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  -----  ------  ----  --------
 0  gini                   6              10                    2  0.999849      0   1.953    49  SYN
 1  gini                   6              10                    3  1             0   1.487     0  HTTP
 2  entropy                3               4                    4  1             0   0.55     27  ACK
 3  gini                   5              20                    6  1             0   0.88     17  UDP
 4  gini                  17              51                    2  1             0   8.867    35  ARP
 5  entropy                6               6                    2  1             0   0.375    16  SP
 6  entropy                6              21                    3  1             0   0.933    37  BF
 7  entropy                1              53                    4  1       

# RandomizedSearchCV RF

In [27]:
lines=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]



for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))



    # use a full grid over all parameters
    param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                  "n_estimators" : sp_randint(1, 200),
                  "max_features": sp_randint(1, 11),
                  "min_samples_split":sp_randint(2, 11),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    second=time()
    f1=[]
    clf=RandomForestClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("RF_HPO.csv",index=False)

final_parametres=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9998489480892209   0.987    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:31<00:00, 21.14s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.571    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [07:20<00:00, 44.07s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.861    0       


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.855    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:07<00:00, 18.70s/it]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.768    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:28<00:00, 14.83s/it]

    bootst    criter      max_depth    max_features    min_samp_split    n_estimators        F1    Std     Time    No  Attack
--  --------  --------  -----------  --------------  ----------------  --------------  --------  -----  -------  ----  --------
 0  False     gini                7               8                 2              98  0.999849      0   24.569     2  SYN
 1  True      gini                7               9                 2              73  1             0   39.185     1  HTTP
 2  True      entropy             8               7                 8              55  1             0   21.485     4  ACK
 3  False     entropy            14               3                 3              35  1             0   14.393     7  UDP
 4  False     gini               17               9                 2             140  1             0  140.288     5  ARP
 5  False     gini                8               7                 2               1  1             0    9.688     6  SP
 6  Fals

# XGBClassifier

In [8]:
import xgboost

lines=[['n_estimators', 'min_child_weight', 'max_depth','learning_rate', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    


    param_grid =   {
    'n_estimators': [100, 500, 900, 1100, 1500],
    'max_depth': [2, 3, 5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.15, 0.20],
    'min_child_weight': [1, 2, 3, 4]
    }

    second=time()
    f1=[]
    clf=xgboost.XGBClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(5)):
        second=time()
        a,b,clf=run_random_search(xgboost.XGBClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("XGB_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
default                             0.9996979037876128   1.158    0       


 20%|████████████████▌                                                                  | 1/5 [02:32<10:10, 152.57s/it]

{'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 15, 'learning_rate': 0.2}       0.9996979037876128   152.574  0       


 40%|█████████████████████████████████▏                                                 | 2/5 [03:56<05:36, 112.03s/it]

{'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.15}        0.9996979037876128   83.638   1       


...

 80%|████████████████████████████████████████████████████████████████████████████████████████████████                        | 4/5 [02:28<00:38, 38.58s/it]

{'n_estimators': 1100, 'min_child_weight': 2, 'max_depth': 2, 'learning_rate': 0.15}       1.0                  39.524   3       


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:13<00:00, 38.74s/it]

{'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.15}       1.0                  45.325   4       
      n_estimators    min_child_weight    max_depth    learning_rate        F1    Std     Time    No  Attack
--  --------------  ------------------  -----------  ---------------  --------  -----  -------  ----  --------
 0            1500                   1           15             0.2   0.999698      0  152.574     0  SYN
 1             500                   1            5             0.15  0.999698      0   83.642     1  SYN
 2            1100                   3           10             0.2   0.999698      0  123.905     2  SYN
 3            1500                   4            2             0.2   0.999698      0  142.78      3  SYN
 4            1100                   4           10             0.2   0.999698      0  132.291     4  SYN
 5             900                   3            2             0.15  1             0   56.532     0  HTTP
 6           

NameError: name 'final_parametres' is not defined

In [9]:
final_parametres=[['n_estimators', 'max_depth', 'learning_rate','min_child_weight', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

      n_estimators    max_depth    learning_rate    min_child_weight        F1    Std     Time    No  Attack
--  --------------  -----------  ---------------  ------------------  --------  -----  -------  ----  --------
 0            1500            4                2                0.2   0.999698      0  142.78      3  SYN
 1             900            3                2                0.15  1             0   56.532     0  HTTP
 2            1100            1                2                0.15  1             0  208.268     4  ACK
 3             900            2                2                0.15  1             0  265.623     1  UDP
 4            1100            3                2                0.2   1             0  762.956     2  ARP
 5            1100            1                3                0.2   0.999774      0   21.275     0  SP
 6            1100            4                2                0.05  1             0   29.196     2  BF
 7             100            3        

# RandomizedSearchCV  KNeighborsClassifier

In [10]:
lines=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    

    # use a full grid over all parameters
    param_grid = {"n_neighbors" : sp_randint(1,64) ,  
                 "leaf_size": sp_randint(1,50) , 
                  "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"],
                  "weights" : ["uniform", "distance"]}
    second=time()
    f1=[]
    clf=KNeighborsClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(KNeighborsClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("knn_HPO.csv",index=False)

final_parametres=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["n_neighbors"].min()
    df=df[df["n_neighbors"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    

SYN
default                             0.9993958379274827   76.132   24      


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [10:21<00:00, 62.10s/it]


HTTP
default                             0.991847328641079    56.955   24      


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:30<00:00, 21.05s/it]


ACK
default                             0.9996159754224268   23.124   24      


...

SCHD
default                             0.9994632280232235   24.859   24      


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:55<00:00, 11.52s/it]


MHDis
default                             0.9846632412684879   20.63    24      


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:40<00:00, 10.03s/it]

    algorithm      leaf_size    n_neighbors  weights          F1    Std    Time    No  Attack
--  -----------  -----------  -------------  ---------  --------  -----  ------  ----  --------
 0  kd_tree               27             11  distance   0.999849      0  71.841     8  SYN
 1  ball_tree             40              2  distance   1             0  22.803     4  HTTP
 2  auto                  13              1  distance   1             0  14.424     3  ACK
 3  ball_tree             45              9  uniform    1             0  79.638     5  UDP
 4  kd_tree               10              2  distance   1             0  33.847     1  ARP
 5  auto                  33              1  distance   1             0   4.55      0  SP
 6  auto                  10              9  distance   1             0  16.7       2  BF
 7  brute                  7             10  distance   1             0  16.87      0  OS
 8  kd_tree               21              1  distance   1             0   8.476     

# GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [12]:
lines=[['var_smoothing', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
        'var_smoothing': np.logspace(0,-9, num=100),
    }

 
    second=time()
    f1=[]
    clf=GaussianNB()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(GaussianNB(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("NB_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.9933067878021322   0.044    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.12it/s]


HTTP
default                             0.6315732269934049   0.047    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.52it/s]


ACK
default                             0.9105941815426182   0.015    0       


...

SCHD
default                             0.866209454746268    0.031    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.81it/s]


MHDis
default                             0.7956485114074345   0.014    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.88it/s]

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      4.32876e-09  0.992101  0             0.824     0  SYN
 1      8.11131e-09  0.991096  0             0.772     1  SYN
 2      6.57933e-09  0.991498  0             0.916     2  SYN
 3      3.51119e-09  0.992402  0             0.76      3  SYN
 4      3.51119e-09  0.992402  0             0.883     4  SYN
 5      5.3367e-09   0.991749  1.11022e-16   0.92      5  SYN
 6      2.31013e-09  0.992854  0             0.94      6  SYN
 7      2.84804e-07  0.975805  0             0.991     7  SYN
 8      1e-08        0.990694  0             0.944     8  SYN
 9      1.23285e-09  0.993206  1.11022e-16   0.959     9  SYN
10      1.87382e-09  0.631096  0             0.659     0  HTTP
11      1e-08        0.628884  0             0.752     1  HTTP
12      1.87382e-09  0.631096  0             0.689     2  HTTP
13      2.84804e-09  0.630567  0             0.619     3  H

In [13]:

final_parametres=[["var_smoothing","F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      1.23285e-09  0.993206  1.11022e-16   0.959     9  SYN
 1      1e-09        0.631573  0             0.652     5  HTTP
 2      2.84804e-07  1         0             0.302     0  ACK
 3      1.23285e-09  0.67223   0             0.631     6  UDP
 4      1.23285e-08  0.499829  0             1.23      9  ARP
 5      1e-09        0.976647  1.11022e-16   0.207     2  SP
 6      0.053367     0.716004  0             0.409     2  BF
 7      1e-09        0.791791  0             0.67      2  OS
 8      0.00432876   0.935426  0             0.522     2  SCHD
 9      1e-09        0.795649  0             0.347     0  MHDis


# LR

In [14]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
     'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' : ['none', 'l1', 'l2', 'elasticnet'],
    'C' : loguniform(1e-5, 100)
        }
    

 
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(LogisticRegression(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

SYN
default                             0.9988924618182504   0.294    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:34<00:00,  3.41s/it]


HTTP
default                             1.0                  0.453    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:34<00:00,  9.48s/it]


ACK
default                             1.0                  0.104    0       


...

SCHD
default                             1.0                  0.383    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.51s/it]


MHDis
default                             1.0                  0.524    0       


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.31s/it]

               C  penalty    Solver           F1          Std     Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  -------  ----  --------
 0   0.000226652  none       lbfgs      0.998892  1.11022e-16    3.965     0  SYN
 1  59.8441       l1         liblinear  0.999708  2.01389e-05    1.851     1  SYN
 2   0.00752025   none       lbfgs      0.998892  1.11022e-16    4.067     2  SYN
 3   0.0373914    l1         liblinear  0.998188  0.000335314    1.763     3  SYN
 4  54.4615       none       lbfgs      0.998892  1.11022e-16    4.474     4  SYN
 5   0.126728     l2         lbfgs      0.998892  1.11022e-16    4.02      5  SYN
 6  68.3723       l1         liblinear  0.999607  4.93285e-05    1.772     6  SYN
 7   0.00140945   none       lbfgs      0.998892  1.11022e-16    4.539     7  SYN
 8   0.33933      none       lbfgs      0.998892  1.11022e-16    3.577     8  SYN
 9   0.281124     none       lbfgs      0.998892  1.11022e-16    4.034     9  SYN
10   0.0

# MLP

In [8]:
import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}


In [9]:
# MLP

import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}




lines=[['solver', 'learning_rate','hidden_layer_sizes','alpha','activation', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    

    param_grid =  {
        'hidden_layer_sizes': layers,
        'solver': ['sgd', 'adam', 'lbfgs'],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001,0.001,0.01,0.1, 0.05],
        'learning_rate': ['constant','adaptive']
    }



    second=time()
    f1=[]
    clf=MLPClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(MLPClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("ann_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.9992447910957365   4.668    0       


100%|███████████████████████████████████████████| 1/1 [32:10<00:00, 1930.51s/it]

{'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (1184, 433), 'alpha': 0.05, 'activation': 'relu'} 0.9971302541324346   1930.511 0       
HTTP


default                             0.9916266889370977   2.903    0       


100%|████████████████████████████████████████████| 1/1 [07:44<00:00, 464.25s/it]

...

100%|███████████████████████████████████████████| 1/1 [19:57<00:00, 1197.30s/it]

{'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (310, 1663), 'alpha': 0.1, 'activation': 'tanh'} 1.0                  1197.298 0       
MHDis


default                             1.0                  1.55     0       


100%|███████████████████████████████████████████| 1/1 [26:55<00:00, 1615.16s/it]

{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (1568, 1928), 'alpha': 0.01, 'activation': 'tanh'} 0.9987873502938804   1615.159 0       
    solver    learning_rate    hidden_layer_sizes      alpha  activation          F1    Std      Time    No  Attack
--  --------  ---------------  --------------------  -------  ------------  --------  -----  --------  ----  --------
 0  adam      adaptive         (1184, 433)            0.05    relu          0.99713       0  1930.51      0  SYN
 1  adam      adaptive         (439,)                 0.0001  relu          0.99607       0   464.244     0  HTTP
 2  lbfgs     adaptive         (1193, 630)            0.05    tanh          1             0  1364.7       0  ACK
 3  adam      adaptive         (1391, 1912)           0.0001  relu          0.999423      0  2493.75      0  UDP
 4  adam      constant         (1015,)                0.1     relu          0.797628      0   796.353     0  ARP
 5  adam      constant         (851, 318

-------------

# RandomizedSearchCV  SVM

In [10]:
lines=[[ 'gamma','C', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    


    param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}  

    second=time()
    f1=[]
    clf=svm.SVC()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("svm_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.9258933118713963   14.975   0       


100%|████████████████████████████████████████████| 1/1 [08:57<00:00, 537.67s/it]

{'gamma': 1, 'C': 10}                                                                      0.9998489480892209   537.668  0       
HTTP


default                             0.4897829026250669   3.253    0       


100%|████████████████████████████████████████████| 1/1 [10:42<00:00, 642.06s/it]

...

100%|█████████████████████████████████████████████| 1/1 [01:32<00:00, 92.68s/it]

{'gamma': 1, 'C': 10}                                                                      1.0                  92.677   0       
MHDis


default                             0.49480455220188024  0.422    0       


100%|████████████████████████████████████████████| 1/1 [02:28<00:00, 148.37s/it]

{'gamma': 0.1, 'C': 1}                                                                     1.0                  148.366  0       
      gamma    C        F1    Std      Time    No  Attack
--  -------  ---  --------  -----  --------  ----  --------
 0     1      10  0.999849      0   537.668     0  SYN
 1     1       1  1             0   642.06      0  HTTP
 2     1      10  1             0   207.411     0  ACK
 3     0.1    10  1             0    59.592     0  UDP
 4     0.01    1  1             0  1529.95      0  ARP
 5     1       1  1             0   118.961     0  SP
 6     0.01   10  1             0   101.159     0  BF
 7     0.01   10  1             0   303.606     0  OS
 8     1      10  1             0    92.677     0  SCHD
 9     0.1     1  1             0   148.366     0  MHDis


In [ ]:
import xgboost